In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
df = pd.read_csv("/kaggle/input/dupla2/dupla2.csv", na_values = [' ?', '?', '? '])
df_cleaned = df.dropna()

In [3]:
# Identifica as colunas numéricas para normalização
colunas_numericas = ['fnlwgt', 'capital-gain', 'capital-loss']

# Identifica colunas categóricas para one-hot encoding (excluding 'earns')
colunas_categoricas_original = df_cleaned.select_dtypes(include=['object']).columns.tolist()
colunas_categoricas_original.remove('earns')
colunas_categoricas_original.remove('sex')
print(f"Colunas categóricas identificadas para one-hot encoding: {colunas_categoricas_original}")

Colunas categóricas identificadas para one-hot encoding: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']


In [4]:
# Aplica one-hot encoding nas colunas categóricas
df_cleaned_encoded = pd.get_dummies(df_cleaned, columns=colunas_categoricas_original, drop_first=True)
print("Colunas categóricas transformadas usando one-hot encoding.")
print("Novas colunas após one-hot encoding (primeiras linhas):")
print(df_cleaned_encoded.head())

Colunas categóricas transformadas usando one-hot encoding.
Novas colunas após one-hot encoding (primeiras linhas):
   age  fnlwgt  education-num      sex  capital-gain  capital-loss  \
0   39   77516             13     Male          2174             0   
1   50   83311             13     Male             0             0   
2   38  215646              9     Male             0             0   
3   53  234721              7     Male             0             0   
4   28  338409             13   Female             0             0   

   hours-per-week   earns  workclass_ Local-gov  workclass_ Private  ...  \
0              40   <=50K                 False               False  ...   
1              13   <=50K                 False               False  ...   
2              40   <=50K                 False                True  ...   
3              40   <=50K                 False                True  ...   
4              40   <=50K                 False                True  ...   

   nati

In [5]:
# Inicializa o MinMaxScaler
scaler = MinMaxScaler()
print("Normalizador MinMaxScaler inicializado.")

Normalizador MinMaxScaler inicializado.


In [6]:
# Cria uma cópia para normalização
df_normalizado = df_cleaned_encoded.copy()

In [7]:
# Normaliza as colunas numéricas
df_normalizado[colunas_numericas] = scaler.fit_transform(df_cleaned[colunas_numericas])
print(f"Colunas numéricas ({', '.join(colunas_numericas)}) normalizadas usando MinMaxScaler.")

Colunas numéricas (fnlwgt, capital-gain, capital-loss) normalizadas usando MinMaxScaler.


In [8]:
# Codifica a variável alvo 'earns'
# Mapeia '<=50K' para 0 e '>50K' para 1
df_normalizado['output'] = df_cleaned['earns'].apply(lambda val: 1 if val.strip() == '>50K' else 0)
print("Coluna alvo 'earns' codificada para 'output' (0 para <=50K, 1 para >50K).")

label_encoder = preprocessing.LabelEncoder()
df_normalizado['sex'] = label_encoder.fit_transform(df_cleaned['sex'])

Coluna alvo 'earns' codificada para 'output' (0 para <=50K, 1 para >50K).


In [9]:
# Seleciona as colunas de features e a coluna alvo
colunas_features = colunas_numericas
df_entrada_perceptron = df_normalizado[colunas_features + ['output'] + ['sex']]

In [10]:
# --- Modelo Perceptron (conforme fornecido pelo usuário) ---
class Perceptron:
    def __init__(self, learning_rate=0.1, n_iter=100):
        self.learning_rate = learning_rate  # Taxa de aprendizado
        self.n_iter = n_iter                # Número de iterações (épocas)
        self.weights = None                 # Pesos
        self.bias = None                    # Viés (bias)

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        y_array = y.values if isinstance(y, pd.Series) else y

        for _ in range(self.n_iter):
            for idx, x_i in enumerate(X.values if isinstance(X, pd.DataFrame) else X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_function(linear_output)
                
                update = self.learning_rate * (y_array[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update

    def activation_function(self, x):
        return np.where(x >= 0, 1, 0)

    def predict(self, X):
        X_values = X.values if isinstance(X, pd.DataFrame) else X
        linear_output = np.dot(X_values, self.weights) + self.bias
        return self.activation_function(linear_output)

In [11]:
# --- Divisão, Treinamento e Teste ---
X = df_entrada_perceptron[colunas_features]
y = df_entrada_perceptron['output']

In [12]:
# Parâmetros para train_test_split
test_size_param = 0.2
random_state_param = 42
print(f"\nDividindo os dados: test_size={test_size_param}, random_state={random_state_param}.")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_param, random_state=random_state_param)


Dividindo os dados: test_size=0.2, random_state=42.


In [13]:
# Parâmetros do Perceptron
learning_rate_param = 0.01
n_iter_param = 100
print(f"Inicializando Perceptron: learning_rate (eta0)={learning_rate_param}, n_iter (max_iter)={n_iter_param}.")
perceptron = Perceptron(learning_rate=learning_rate_param, n_iter=n_iter_param)

Inicializando Perceptron: learning_rate (eta0)=0.01, n_iter (max_iter)=100.


In [14]:
print("Treinando o modelo Perceptron...")
perceptron.fit(X_train, y_train)
print("Treinamento concluído.")

Treinando o modelo Perceptron...
Treinamento concluído.


In [15]:
print("\nFazendo predições no conjunto de teste...")
predicoes = perceptron.predict(X_test)


Fazendo predições no conjunto de teste...


In [16]:
# --- Exibe os Resultados e Métricas ---
print("\n--- Resultados do Modelo Perceptron ---")

print("\nPesos Aprendidos:")
print(perceptron.weights)
print("\nViés (Bias) Aprendido:")
print(perceptron.bias)


--- Resultados do Modelo Perceptron ---

Pesos Aprendidos:
[7.98539161e-05 1.10647706e-01 1.08585859e-03]

Viés (Bias) Aprendido:
-0.01


In [17]:
# Cálculo das métricas de desempenho
acc = accuracy_score(y_test.values, predicoes)
prec = precision_score(y_test.values, predicoes, zero_division=0)
rec = recall_score(y_test.values, predicoes, zero_division=0)
f1 = f1_score(y_test.values, predicoes, zero_division=0)

In [18]:
print("\n--- Métricas de Desempenho no Conjunto de Teste ---")
print(f"Acurácia: {acc:.4f}")
print(f"Precisão: {prec:.4f}")
print(f"Recall (Sensibilidade): {rec:.4f}")
print(f"Pontuação F1: {f1:.4f}")


--- Métricas de Desempenho no Conjunto de Teste ---
Acurácia: 0.7688
Precisão: 0.9787
Recall (Sensibilidade): 0.0902
Pontuação F1: 0.1652


In [19]:
# Informações sobre os parâmetros utilizados, para confirmar as respostas anteriores
print("\n--- Confirmação dos Parâmetros Utilizados ---")
print(f"Normalização: MinMaxScaler")
print(f"Parâmetros do train_test_split: test_size={test_size_param}, random_state={random_state_param}")
print(f"Parâmetros do Perceptron: n_iter (max_iter)={n_iter_param}, learning_rate (eta0)={learning_rate_param}, random_state=Não utilizado na classe Perceptron fornecida")


--- Confirmação dos Parâmetros Utilizados ---
Normalização: MinMaxScaler
Parâmetros do train_test_split: test_size=0.2, random_state=42
Parâmetros do Perceptron: n_iter (max_iter)=100, learning_rate (eta0)=0.01, random_state=Não utilizado na classe Perceptron fornecida


In [20]:
df_normalizado.to_csv('df_normalizado.csv', index=False)